In [7]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('iwRegisteredCompanies.db')

In [15]:
df1 = pd.read_csv("iwRegisteredCompanies.csv")
df1.to_sql('iwRegisteredCompanies', conn, if_exists='replace', index=False)
df2 = pd.read_csv("sic_codes.csv")
df2.to_sql('sic_codes', conn, if_exists='replace', index=False)

df_exploded = df1.assign(sic_code=df1['nature_of_business'].str.split()).explode('sic_code')
df_exploded['sic_code'] = df_exploded['sic_code'].astype('Int64')
df_exploded[['company_number', 'sic_code']].to_sql('nature_of_business', conn, if_exists='replace', index=False)


4901

In [34]:
# Via https://www.ons.gov.uk/aboutus/transparencyandgovernance/freedomofinformationfoi/siccodesandgroupingnames

sic_hierarchy_df =pd.read_csv("publisheduksicsummaryofstructureworksheet.csv")
for c in ["SECTION","Division","Group"]:
    sic_hierarchy_df[c] = sic_hierarchy_df.apply(lambda row: row["Description"] if row["Level headings"] == c else None, axis=1)
    sic_hierarchy_df[c] = sic_hierarchy_df[c].ffill()
sic_hierarchy_df = sic_hierarchy_df[sic_hierarchy_df["Level headings"]=="Class"]
sic_hierarchy_df.dropna(how="all", inplace=True, axis=1)
sic_hierarchy_df.rename(columns={"SECTION":"Section","Most disaggregated level":"sic_code"}, inplace=True)
cols = ["sic_code", "Class", "Section", "Division", "Group"]
sic_hierarchy_df[cols].to_sql('sic_code_hierarchy', conn, if_exists='replace', index=False)


615